In [12]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from operator import itemgetter

In [13]:
train = pd.read_csv('Updated_frame.csv')
train.head()

,id,gender,area,qualification,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy,...,income_int,risk_factor,no_of_claims,policy_tenure,avg_premium_amount,num_policies_int,claim_amt_to_income_ratio,premium_amt_to_income_ratio,no_of_policies_to_vintage_ratio,no_of_claims_to_vintage_ratio
0,1,1,1,0,1,5,5790,1,0,1,...,75000,0.077200,89392,3,2.542901,2,0.077200,0.000040,0.400000,17878.400000
1,2,1,0,1,0,8,5080,1,0,1,...,75000,0.067733,89392,3,2.542901,2,0.067733,0.000040,0.250000,11174.000000
2,3,1,1,0,1,8,2599,1,0,1,...,75000,0.034653,89392,3,2.542901,2,0.034653,0.000040,0.250000,11174.000000
3,4,0,0,1,0,7,0,1,0,1,...,75000,0.000000,89392,3,2.542901,2,0.000000,0.000040,0.285714,12770.285714
4,5,1,1,1,1,6,3508,1,0,0,...,120000,0.029233,89392,3,2.542901,2,0.029233,0.000025,0.333333,14898.666667


In [16]:
train['num_policies'].unique()

array([1, 0], dtype=int64)

In [9]:
724068 - 31790

692278

In [8]:
train['cltv'].describe()

count     89392.000000
mean      97952.828978
std       90613.814793
min       24828.000000
25%       52836.000000
50%       66396.000000
75%      103440.000000
max      724068.000000
Name: cltv, dtype: float64

In [3]:
train.columns

Index(['id', 'gender', 'area', 'qualification', 'marital_status', 'vintage',
       'claim_amount', 'num_policies', 'policy', 'type_of_policy', 'cltv',
       'age', 'family_size', 'spending_power', 'education_level', 'income_int',
       'risk_factor', 'no_of_claims', 'policy_tenure', 'avg_premium_amount',
       'num_policies_int', 'claim_amt_to_income_ratio',
       'premium_amt_to_income_ratio', 'no_of_policies_to_vintage_ratio',
       'no_of_claims_to_vintage_ratio'],
      dtype='object')

In [6]:
X = train.drop('cltv', axis=1)
y = train['cltv']
X_constant = sm.add_constant(X)
lin_reg=sm.OLS(y, X_constant).fit()
lin_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cltv   R-squared:                       0.149
Model:                            OLS   Adj. R-squared:                  0.149
Method:                 Least Squares   F-statistic:                     981.5
Date:                Sun, 22 Jan 2023   Prob (F-statistic):               0.00
Time:                        19:33:48   Log-Likelihood:            -1.1400e+06
No. Observations:               89392   AIC:                         2.280e+06
Df Residuals:                   89375   BIC:                         2.280e+06
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
id                                 -0.0108      0.011     -0.994      0.320      -0.032       0.010
gender                           2160.9340    568.641      3.800      0.000    1046.403    3275.465
area                             7700.0374    699.468     11.008      0.000    6329.087    9070.988
qualification                    -793.8898    246.564     -3.220      0.001   -1277.152    -310.627
marital_status                  -3353.6875    286.445    -11.708      0.000   -3915.117   -2792.258
vintage                           123.8363    157.400      0.787      0.431    -184.666     432.339
claim_amount                        3.0025      0.139     21.649      0.000       2.731       3.274
num_policies                     3.179e+04    456.327     69.669      0.000    3.09e+04    3.27e+04
policy                            444.9887    251.062      1.772      0.076     -47.090     937.068
type_of_policy                  -1776.6230    412.029     -4.312      0.000   -2584.195    -969.051
age                                 0.3390      0.431      0.787      0.431      -0.506       1.184
family_size                     -3353.6893    286.445    -11.708      0.000   -3915.119   -2792.260
spending_power                     -0.0004      0.001     -0.448      0.654      -0.002       0.002
education_level                   793.8896    246.564      3.220      0.001     310.627    1277.152
income_int                         -0.0454      0.010     -4.731      0.000      -0.064      -0.027
risk_factor                     -7585.6820   4790.876     -1.583      0.113    -1.7e+04    1804.390
no_of_claims                        0.1754      0.022      7.933      0.000       0.132       0.219
policy_tenure                    -444.9887    251.062     -1.772      0.076    -937.068      47.090
avg_premium_amount               3.826e-06   1.76e-06      2.179      0.029    3.85e-07    7.27e-06
num_policies_int                 3.179e+04    456.327     69.669      0.000    3.09e+04    3.27e+04
claim_amt_to_income_ratio       -7585.6820   4790.876     -1.583      0.113    -1.7e+04    1804.390
premium_amt_to_income_ratio      1.731e+07   2.32e+07      0.747      0.455   -2.81e+07    6.27e+07
no_of_policies_to_vintage_ratio  3972.1067   2335.485      1.701      0.089    -605.422    8549.635
no_of_claims_to_vintage_ratio      -0.0620      0.045     -1.379      0.168      -0.150       0.026
==============================================================================
Omnibus:                    51267.841   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           404871.936
Skew:                           2.737   Prob(JB):                         0.00
Kurtosis:                      11.873   Cond. No.                  

In [7]:
X = train.drop('cltv', axis=1)
y = train['cltv']

pd.set_option('display.max_rows', 67)
vif = [variance_inflation_factor(X_constant.values, i) for i in range(X_constant.shape[1])]
pd.DataFrame({'vif': vif}, index=X_constant.columns)

C:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\regression\linear_model.py:1736: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


,vif
id,1.000175
gender,1.017110
area,1.318208
qualification,inf
marital_status,inf
vintage,inf
claim_amount,2.619916
num_policies,inf
policy,inf
type_of_policy,1.010919
